## requirements

In [1]:
# %pip install opencv-python
# %pip install numpy
# %pip install typing
# %pip install scikit-learn

## import libraries

In [2]:
import cv2
import numpy as np
from typing import Tuple, List
import math
from sklearn.cluster import KMeans

## download image

In [3]:
image = cv2.imread('../input_images/road1.png')
image_copy = image.copy()

## image preprocessing

In [4]:
# Converting the original image to HLS
def convert_to_hls(img: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(img, cv2.COLOR_BGR2HLS)


def isolate_white_color(hsl_img: np.ndarray, img: np.ndarray ) -> np.ndarray:
    # Defining the lower and upper threshold for white in HLS
    lower_white = np.array([0, 200, 0], dtype=np.uint8)     # Lower threshold for brightness, average saturation and hue
    upper_white = np.array([255, 255, 255], dtype=np.uint8) # Upper threshold for brightness, saturation and hue

    # Applying a threshold operation to isolate the white color in an HLS image
    white_mask = cv2.inRange(hsl_img, lower_white, upper_white)
    return cv2.bitwise_and(img, img, mask=white_mask)


def gaussianBlur(img: np.ndarray) -> np.ndarray:
    return cv2.GaussianBlur(img, (5, 5), 0)


## getting lines

In [5]:
def getbordersCanny(img: np.ndarray) -> np.ndarray:
    return cv2.Canny(img, 100, 200)

def limit_field_of_view(img: np.ndarray) -> np.ndarray:
    height, width = img.shape[:2]
    
    # Creating a mask for an area of interest
    mask = np.zeros((height, width), dtype=np.uint8)
    
    new_width = int(width) 
    new_height = int(height * 1.5)                  # 1.5 times increase in height
    roi_y = (new_height - height) // 2              # shifting the area of interest down
    roi_x = (width - new_width) // 2                # shifting the area of interest to the right
    roi_corners = np.array([[(roi_x + 520, roi_y),  # offset from the left side by 520 pixels
                            (width - roi_x, roi_y), 
                            (width - roi_x, new_height), 
                            (roi_x, new_height)]], dtype=np.int32) 

    cv2.fillPoly(mask, roi_corners, 255)

    return cv2.bitwise_and(img, img, mask=mask)

def get_Huff_lines(img: np.ndarray) -> np.ndarray:
    return cv2.HoughLinesP(img, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=20)
    

## removing unnecessary lines

<b>Note:</b>

The resultant set of lines is partitioned into two clusters, distinguished based on their affiliation with the markings on the right and left sides of the road. Following this, within each cluster, the central line is determined by calculating the mean coordinates of the lines within that specific cluster.

In [6]:

def extract_features(lines: np.ndarray) -> np.ndarray:
    features = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        angle = np.arctan2(y2 - y1, x2 - x1)
        features.append([x1, y1, x2, y2, length, angle])
    return np.array(features)

def get_clusters_lines(lines: np.ndarray) -> list:
    kmeans = KMeans(n_clusters=2) 
    features = extract_features(lines)
    kmeans.fit(features)
    clusters = [[] for _ in range(kmeans.n_clusters)]

    for i, line in enumerate(lines):
        cluster_idx = kmeans.labels_[i] 
        clusters[cluster_idx].append(line)
    
    return clusters


In [7]:
def find_middle_line(lines: list) -> list:
    # сalculation of the average coordinates of the start and end points for all lines
    start_points = np.mean([line[0][:2] for line in lines], axis=0)
    end_points = np.mean([line[0][2:] for line in lines], axis=0)
    
    # сalculating the midpoint for all lines
    middle_point = tuple(((start_points + end_points) / 2).astype(int))

    # сalculating the angle of inclination of the midline
    angle = np.arctan2(end_points[1] - start_points[1], end_points[0] - start_points[0])

    length = 280  
    # starting point
    x1 = middle_point[0] - int(length * np.cos(angle))
    y1 = middle_point[1] - int(length * np.sin(angle))

    # the end point
    x2 = middle_point[0] + int(length * np.cos(angle))
    y2 = middle_point[1] + int(length * np.sin(angle))

    return [x1, y1, x2, y2]

def get_center_lines(clusters: list ) -> list:
    center_lines = []
    for cluster in clusters:
        center_lines.append(find_middle_line(cluster))
    return center_lines


## finding an intersection point

In [8]:
def find_intersection(center_lines: list) -> Tuple[int, int]:
    x1, y1, x2, y2 = center_lines[0]
    x3, y3, x4, y4 = center_lines[1]

    # сalculation of angular coefficients of straight lines
    m1 = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else float('inf')
    m2 = (y4 - y3) / (x4 - x3) if (x4 - x3) != 0 else float('inf')

    # check the parallelism of the lines
    if m1 == m2:
        return None  

    # сalculating the coordinates of the intersection point
    if m1 == float('inf'):
        x = x1
        y = m2 * (x - x3) + y3
    elif m2 == float('inf'):
        x = x3
        y = m1 * (x - x1) + y1
    else:
        x = ((m1 * x1 - y1) - (m2 * x3 - y3)) / (m1 - m2)
        y = m1 * (x - x1) + y1

    return int(x), int(y)

## presentation of results 

In [9]:
def draw_result(img: np.ndarray, lines: list, intersection_point: Tuple[int, int]) -> None:
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line
            cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 10)

    cv2.circle(img, tuple(intersection_point), 15, (0, 165, 255), -1)

def save_image(img: np.ndarray, name: str) -> None:
    cv2.imwrite(name, img)
    

## pipeline of work

In [10]:
result_image = convert_to_hls(image_copy)
result_image = isolate_white_color(result_image, image_copy)
result_image = gaussianBlur(result_image)
result_image = getbordersCanny(result_image)
result_image = limit_field_of_view(result_image)
result_lines = get_Huff_lines(result_image)

clusters = get_clusters_lines(result_lines)

center_lines = get_center_lines(clusters)
intersection_point = find_intersection(center_lines)

draw_result(image, center_lines, intersection_point)
save_image(image, '../output_images/result_road1.png')

In [11]:
image_origin = cv2.imread('../input_images/road1.png')
combined_image = cv2.hconcat([image_origin, image])
save_image(combined_image, '../output_images/result_road1_combined.png')